In [13]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [14]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [16]:
# define the NN by subclassing nn.Module.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [17]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [18]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) # pass it some random input data
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='mps:0')


In [19]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [20]:
# flatten the input image
flatten = nn.Flatten()
flat_image = flatten(input_image)
flat_image.size()

torch.Size([3, 784])

In [22]:
# The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [24]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[0.0000, 0.2444, 0.0000, 0.0614, 0.3624, 0.1988, 0.3124, 0.1201, 0.5258,
         0.8654, 0.3689, 0.0000, 0.2408, 0.0147, 0.0000, 0.2005, 0.0000, 0.2627,
         0.5090, 0.2774],
        [0.0000, 0.0000, 0.5095, 0.1898, 0.4887, 0.0000, 0.0436, 0.0000, 0.3948,
         0.7132, 0.0995, 0.0000, 0.0318, 0.4332, 0.2952, 0.1551, 0.0298, 0.1432,
         0.3211, 0.2028],
        [0.2440, 0.3671, 0.4657, 0.1987, 0.1407, 0.0000, 0.2443, 0.1953, 0.5376,
         0.8488, 0.3239, 0.0000, 0.0000, 0.2451, 0.0000, 0.1743, 0.0135, 0.0859,
         0.4328, 0.2283]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.0000, 0.2444, 0.0000, 0.0614, 0.3624, 0.1988, 0.3124, 0.1201, 0.5258,
         0.8654, 0.3689, 0.0000, 0.2408, 0.0147, 0.0000, 0.2005, 0.0000, 0.2627,
         0.5090, 0.2774],
        [0.0000, 0.0000, 0.5095, 0.1898, 0.4887, 0.0000, 0.0436, 0.0000, 0.3948,
         0.7132, 0.0995, 0.0000, 0.0318, 0.4332, 0.2952, 0.1551, 0.0298, 0.1432,
         0.3211, 0.2028],
       

In [28]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits

tensor([[ 0.2282, -0.0527, -0.1186, -0.1516,  0.1512, -0.2683,  0.1160, -0.1397,
          0.0179,  0.0063],
        [ 0.3164, -0.1189, -0.1200, -0.0015,  0.1750, -0.4008,  0.0502, -0.0174,
         -0.1583, -0.0401],
        [ 0.3710, -0.0953, -0.0272, -0.0367,  0.0030, -0.3029,  0.0360,  0.0319,
         -0.0731, -0.1092]], grad_fn=<AddmmBackward0>)

In [29]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.1269, 0.0958, 0.0897, 0.0868, 0.1175, 0.0773, 0.1135, 0.0879, 0.1029,
         0.1017],
        [0.1392, 0.0901, 0.0900, 0.1013, 0.1209, 0.0680, 0.1067, 0.0997, 0.0866,
         0.0975],
        [0.1459, 0.0915, 0.0980, 0.0971, 0.1010, 0.0744, 0.1044, 0.1039, 0.0936,
         0.0903]], grad_fn=<SoftmaxBackward0>)

In [30]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)




/Users/lapuerta/mambaforge/envs/torchenv/lib/python3.10/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525498485/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0307, -0.0325,  0.0310,  ..., -0.0322, -0.0048,  0.0226],
        [ 0.0150,  0.0266, -0.0354,  ..., -0.0122, -0.0215, -0.0064]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0318, -0.0096], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0145,  0.0165,  0.0162,  ...,  0.0405,  0.0299,  0.0006],
        [ 0.0194, -0.0121,  0.0034,  ..., -0.0272,  0.0027, -0.0105]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([ 0.0037, -0.0106], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[ 0.0367, -0.0295,  0.0153,  ..., -0.0190,  0.0384, -0.0010],
        [-0.0064, -0.0285, -0.0099,  ..

In [38]:
# Create a list of 4 tensors with the same shape
tensors = [torch.ones(2, 3), torch.zeros(2, 3), torch.ones(2, 3), torch.rand(2, 3)]
tensors[0].shape, tensors[1].shape, tensors[2].shape, tensors[3].shape

(torch.Size([2, 3]),
 torch.Size([2, 3]),
 torch.Size([2, 3]),
 torch.Size([2, 3]))

In [39]:
# Concatenate the tensors along a new dimension using stack
stacked = torch.stack(tensors)

print(stacked.shape)  # Output: torch.Size([3, 2, 3])

torch.Size([4, 2, 3])


In [40]:
stacked

tensor([[[1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000]],

        [[0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000]],

        [[0.0560, 0.2154, 0.6622],
         [0.5136, 0.3051, 0.0882]]])

In [43]:
stacked2 = torch.stack(tensors, dim=1)
stacked2.shape

torch.Size([2, 4, 3])

In [44]:
stacked2

tensor([[[1.0000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000],
         [1.0000, 1.0000, 1.0000],
         [0.0560, 0.2154, 0.6622]],

        [[1.0000, 1.0000, 1.0000],
         [0.0000, 0.0000, 0.0000],
         [1.0000, 1.0000, 1.0000],
         [0.5136, 0.3051, 0.0882]]])